In [1]:
import pandas as pd
import requests
import time
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import seaborn as sns
import boto3
from sklearn.cluster import DBSCAN
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

#### Visu de l'historique des feux

In [2]:
feux = pd.read_csv('https://projet-incendie.s3.eu-west-3.amazonaws.com/historique_incendies_avec_coordonnees.csv', sep=';', encoding='utf-8')
pd.set_option('display.max_columns', None)




In [3]:
pd.set_option('display.max_rows', None)

In [4]:
#ajout du code postale via le code insee pour joindre le fichier corse pr la suite
df = pd.read_csv('correspondance-code-insee-code-postal.csv', sep=';', encoding='utf-8')
df = df.drop(columns=['Département','Région','Statut','Altitude Moyenne','Superficie','Population','geo_shape','ID Geofla','Code Commune','Code Canton','Code Arrondissement','Code Département','Code Région'], axis=1)
df.head()

,Code INSEE,Code Postal,Commune,geo_point_2d
0,66033,66300,CAMELAS,"42.639460715710925, 2.6889148905394613"
1,64254,64370,HAGETAUBIN,"43.510561909594024, -0.612895161323306"
2,47260,47270,SAINT-MAURIN,"44.21584680376108, 0.8936370457330151"
3,80118,80150,BOUFFLERS,"50.25894479050434, 2.021288658965274"
4,46147,46220,LAGARDELLE,"44.48709070580215, 1.170693334968643"


In [5]:
df_feux = pd.merge(feux, df, on=['Code INSEE'], how='left')


#### Rajout de la météo

In [7]:
df_meteo=pd.read_csv('output/corse_meteo_cleaned_features.csv', sep=';', encoding='utf-8')


In [ ]:
df_meteo.rename({'DATE': 'Date'}, axis=1, inplace=True)
df_meteo['Date'] = pd.to_datetime(df_meteo['Date']).dt.normalize()
df_feux['Date'] = pd.to_datetime(df_feux['Date']).dt.normalize()

### Merge Feux et météo

In [ ]:
feux_corse = df_feux[df_feux['Département'].isin(['2A', '2B', 2])]
feux_corse = feux_corse.drop(feux_corse.columns[[12, 13, 14, 21]], axis=1)
feux_corse = feux_corse.rename(columns={'Nom de la commune': 'ville'})

In [ ]:
df= pd.merge(df_meteo, feux_corse, on=['Date', 'ville'], how='outer')
df['Feux'].value_counts()

# Test pour ajouter un feu sur bonifacio

In [17]:
# Modifier la valeur pour la ville 'Afa' à la date '2023-10-01'
# df.loc[(df['ville'] == 'Bonifacio') & (df['Date'] == '2025-06-02'), 'Feux'] = 1

### Traitement des doublons

In [18]:
# df['Code Postal'] = df['Code Postal'].astype(str)
df_clean = df.groupby(['ville', 'Date'], as_index=False).agg(lambda x: x.dropna().iloc[0] if not x.dropna().empty else None)



In [24]:
df_clean['Feux'] = df_clean['Feux'].fillna(0)

In [25]:
df_clean['Feux'].value_counts()

Feux
0.0    248927
1.0      8906
Name: count, dtype: int64

In [26]:
df = df_clean

In [27]:
df.to_csv("output/corse_Dataset_modele.csv", index=False, sep=';')



In [28]:
# print("Bucket utilisé :", bucket)
# s3 = boto3.client(
#     "s3",
#     aws_access_key_id=aws_access_key_id,
#     aws_secret_access_key=aws_secret_access_key,
#     region_name=region
# )
# s3.upload_file("Dataset_modele.csv", bucket, "Dataset_modele.csv")